# Case Study: Contrail Segmentation 
## On the Importance of Understanding Your Data and Task
*Rolls off the tongue, doesn't it?* - The Authors of This Notebook

This notebook presents a set of exercises in the form of a case study on segmenting contrails from satellite image (also referred to as remote sensing data). 

*Prerequisties*:
- Basic machine learning knowledge 
- Basic familiarity with the paper "Few-Shot Contrail Segmentation in Remote Sensing Imagery With Loss Function in Hough Space" by Junzi et al. (https://ieeexplore.ieee.org/document/10820969). We do not expect you to understand the whole paper, but rather to have an idea of what is being done.

*The goal by the end of this notebook is for students to:*
- Gain an exposure to a new application of artificial intelligence - extracing semantics from satellite images 
- Understand when they can consider the specifics of their data and task
- Gain exposure to methods to take advantage of their task
- Understand how to evaluate machine learning tasks based on the goal they are solving 
- Go beyond the technical understanding of the problem and consider where this application will be used, will it even be helpful, etc.

## Introduction: Contail Segmentation
The paper "Few-Shot Contrail Segmentation in Remote Sensing Imagery With Loss Function in Hough Space" by Junzi et al. focuses on creating an automatic segmentation procedure for contrails when using few data samples by taking advantage of what we know about the problem. In this section, the problem is introduced from the very basics of what a contrails is, to a mathematical formulation of the issue. 

<p align="center">
  <img src="images/what-is-a-contrail.png" width="800"/>
</p>

*What is a contrail?* 

Contrails or vapour trails are line-shaped clouds produced by aircraft engine exhaust or changes in air pressure, typically at aircraft cruising altitudes several kilometres/miles above the Earth's surface. (Definition - https://en.wikipedia.org/wiki/Contrail)

*Why does it matter?* 

The Sun emits solar radiation towards the Earth that the ground traditionally reflects back. However, the formation of the ice crystals in contrails creates a dense enough "shield" to reflect a part of them back. This impacts the amount of radiation trapped around the Earth and thus contributes to the temperature. They also have a potential cooling effect on sun rays getting reflected back towards the Sun, though this effect is currently estimated to be smaller.


**Satellite Image** 

Satellite images are a unique form of data. As the name implies, they are taken by a satellite, which leads to charactersitics about it, such as:
- **top-down**: the perspective of the image is always orthogonal towards the ground 
- **distance from the Earth**: Depending on the image, and satellite both the distance to the Earth and the resolution of the image is different. 
    - For this reason, the resolution is measured in terms of actual distance (e.g. A resolution of 30cm means that 30cm of information is encoded per pixel.)
    - Common resolutions are 30cm, 1m, 2m. Correspondingly a smaller distance corresponds to higher quality, more expense, harder to get, etc.
- **multispectral data**: Sensory information from satellites is more advanced than traditional cameras. They can capture more than just the color spectrum. Each one is a special band (channel in images). Below are listed some examples: 
    - RGB (Red-Green-Blue): The channels traditional cameras capture, and are blended together to create a final image
    - Infrared/Near-Infrared: A channel where some surfaces react differently to - for example, vegetation reacts differently to infrared light. Can reveal new information, not visible to the naked eye.
    - Point Clouds (LiDAR): A channel capturing distance to the earth. Used to map out things like elevation via the laser reflecting back. 
    - And many others, depending on the sensor used ... We will introduce them in this notebook if and as necessary.

<div style="display: flex; justify-content: space-around;">
   <img src="images/remote-sensing-platforms.webp" width="450"/>

</div>

[Image Source: GeeksForGeeks](https://imgs.search.brave.com/ebFlnbhNCKYFr760b0JRrC-BQlpJUeTmlB5SUw-5Nf4/rs:fit:860:0:0:0/g:ce/aHR0cHM6Ly9tZWRp/YS5nZWVrc2ZvcmdZWtzLm9yZy93cC)




Below are listed some example images of contrail images taken from different satellites and bands: 


<div style="display: flex; justify-content: space-around;">
  <img src="images/1-MeteoSat 11.png" width="350"/>
  <img src="images/2-NASA Terra.jpg" width="350"/>
  <img src="images/3-NOAA Suomi-NPP.jpg" width="350"/>
</div>

[Image Source TBD](TBD)


## Project Setup

### Create an Anaconda Environment and Download Requirements
If you haven't already setup the codebase following the instructions of the README, you can do so here. Otherwise, you can skip running this.

In [ ]:
%conda create -y -n contrail-project python=3.12
%conda activate contrail-project
%conda install -y pip
%pip install -r requirements.txt

### Get Imports

In [ ]:
import os
import random
import numpy as np
import torch

def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

    # Force deterministic behavior in cudnn (might slow things down)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed()

## What is Contrail Segmentation?

What are we trying to accomplish? From the name **contrail segmentation**, and the information beforehand, we know that we are trying to extract information out of the satellite images. To do so, we need to define **segmentation** first.

#### Segmentation

Recall **classification**, abstracted from any machine learning model. Given an input $x$, we want to get an output $\widehat y$ corresponding to the correct class $y$ of the object $x$. 

In remote sensing, our inputs are images. Therefore $x \in \mathbb R^{W \times H \times C} $ where $W, H$ are the height and width (in pixels) of the current image and $C$ the number of channels total the image (recall RGB or Infrared introduced previously). Mathematically, this is a **tensor** (it is not very important for today, but good to know in general). 

In **segmentation**, we want to semantically understand our image. A way to do this is to somehow classify what we are seeing in our image. One way to do this is to output another image, classifying each pixel, whether it belongs to the object/s we are looking for. This can be defined as creating a function $f:x \in \mathbb R^{W \times H \times C} \to y \in \mathbb Z_n$ where $n$ is the number of classes we have in our image. An illustration is provided below. Specifically, this is a case called **semantic segmentation**.


<div style="display: flex; justify-content: space-around;">
   <img src="images/semantic_segmentation.jpg" width="750"/>
</div>

[Image Source](https://www.google.com/url?sa=i&url=https%3A%2F%2Fwww.hitechbpo.com%2Fblog%2Fsemantic-segmentation-guide.php&psig=AOvVaw2xoq84cC-FovXhPj2KUpDB&ust=1752526997439000&source=images&cd=vfe&opi=89978449&ved=0CBQQjRxqFwoTCNDgz_7duo4DFQAAAAAdAAAAABAE)


The case of **contrail segmentation** can then be examined as just a subset of **semantic segmentation** where our classes are $n=2$ - "contrail" and "background". Illustration of $x$ and $y$ below:

<div style="display: flex; justify-content: space-around;">
   <img src="contrail-seg/data/goes/florida/image/florida_2020_03_05_0101.png" width="600"/>
   <img src="contrail-seg/data/goes/florida/mask/florida_2020_03_05_0101.png" width="600"/>
</div>

[Image Source TBD](TBD)

This is a challenging task. Throughout the study, the authors present ways they try to compensate for their lack of data (only around 30 images). **This is a common occurence in the ML industry**. In the following sections, you will go through exercises examining each of their approaches, try to add to them and apply them to a different case, and reason about if this is a correct application.

## How Do We Know Our Model Is Good? - Metrics

## Can We Train Our Model Better? - Other Loss Functions

## Data Scarcity - Data Augmentation

Scarcity is an issue everybody who works with data faces eventually, even in foundational work - https://www.eecis.udel.edu/~shatkay/Course/papers/NetworksAndCNNClasifiersIntroVapnik95.pdf. **Data Augmentation** is a way to reduce overfitting on incomplete datasets. 

Intuitively, it introduces distortions to the small data samples, which vary during training so that the machine learning model of choice does not focus on arbitrary aspects of the data (such as the position of the object, such as a specific position or number). 

Mathematically, it changes the sampling from the training data set $X_{train}$ by a  affine transformation function $g: \mathbb R^n \to \mathbb R^n $, which is applied on every sample point $x \in X$ and outputs a different transformation each time. This change effetively increases the diversity in the training set distribution and brings the final estimate closer to the maximum likelihood on the complete data set $X$ where $X_{train} \subseteq X$. 

*Example* 

 Imagine a scenario where we have a small training set on flowers (such as the Iris dataset), where all daisies and poppies in the training set have a sepal width $s$ of $5$ and $7$ centimeters,  respectively. The test set, and correspondingly all real examples may have no such examples and there are daisies and poppies of many more sizes. A data augmentations technique may be making $g$ apply noise to the inputs, putting the daisies and poppies's $s$ within some standard deviation giving a random variable over the dataset $$s_{\text{augmented}} = s + \varepsilon, \quad \varepsilon \sim \mathcal N(0, \sigma^2)$$

 For instance, daisies might be sampled with sepal widths $5 \pm 0.5$. 


## Colab Setup

In [ ]:
from getpass import getpass
import os

# Securely input your token
token = getpass('Paste your GitLab token: ')

# Set the repository URL
repo_url = "https://oauth2:" + token + "@gitlab.ewi.tudelft.nl/tjviering/contrails.git"

# Clone the repository
!git clone --branch data-augmentation --single-branch {repo_url}
# 9WrCsxUVdPYmy7uij2Fn -Token
# Verify clone
!ls contrails  # Should show repository contents
os.chdir("contrails")
%pip install -r requirements.txt

**This section introduces data augmentation in two modalities: audio and images. Later, we will also show the value of that on our task of contrails.**

One is a fully-guided example, the other ones need to have either code filled in and/or theoretical questions answered. Finally, this is all related to the contrail case. 

### Audio

Here we show a simple task: audio classification. Given an audio file, can we understand automatically what number is being said based on the recording? For this purpose, we are going to use a subset of the AudioMNIST dataset, and evaluate performance on it both with and without data augmentation. 

https://github.com/soerenab/AudioMNIST - AudioMNIST


#### Load Data

In [ ]:
from sklearn.model_selection import train_test_split
from data_util.AudioMNIST.load_data import load_audio_data, load_audio_data_noisy

max_samples = 100
X_audio, y_audio, wavs = load_audio_data('data_util/AudioMNIST/free-spoken-digit-dataset-master/recordings', max_samples=max_samples)
X_audio_augmented, y_audio_augmented, wavs_augmented = load_audio_data_noisy('data_util/AudioMNIST/free-spoken-digit-dataset-master/recordings', max_samples=max_samples)

X_audio_train, X_audio_test, y_audio_train, y_audio_test = train_test_split(X_audio, y_audio, test_size=0.2, random_state=42)
X_audio_train_augmented, _, y_audio_train_augmented, _ = train_test_split(X_audio_augmented, y_audio_augmented, test_size=0.2, random_state=42)

#### Original Audio Samples

An audio file is created when an analog processor converts sound waves into a sequence of amplitudes and spectrograms. They respectively give an idea of when the volume and frequency are higher. When it is played, it is also just an array of amplitudes, showing the intensity. By looking at the waveform graphs, you can clearly discern when someone is speaking, and also play the audio.

In [ ]:
from visualization.show_audio import visualize_waveform_and_play_audio
visualize_waveform_and_play_audio(y_audio[:5], wavs[:5])

#### Augmented Audio Samples

This signal seems to have lots of amplitudes, right, and varying length, based on the person, voice, etc. We have prepared an MNIST dataset, of numbers. However, if we change certain things about the signal, would it necessarily change the label. For example:
- speeding up or slowing down the speech still gives the same number 
- shifting the pitch as well
- adding noise, making a worse recording should not change the outcome as well

This is all a form of **data augmentation** - changes to our data that does not impact how our model should treat it, while being more adversarial and challenging for it. Below we have prepared one of the examples previously listed - adding Gaussian noise. Take a listen and see if the numbers are still discernable.

In [ ]:
visualize_waveform_and_play_audio(y_audio_augmented[:5], wavs_augmented[:5])

#### Training the Models

In [ ]:
from data_util.AudioMNIST.split_data import get_balanced_indices_by_targets

subset_size = 500
samples_per_class = subset_size // 10

balanced_indices = get_balanced_indices_by_targets(y_audio_train, samples_per_class, n_classes=10)
X_audio_train_subset = X_audio_train[balanced_indices]
y_audio_train_subset = y_audio_train[balanced_indices]
balanced_indices = get_balanced_indices_by_targets(y_audio_train_augmented, samples_per_class, n_classes=10)
X_audio_train_augmented_subset = X_audio_train_augmented[balanced_indices]
y_audio_train_augmented_subset = y_audio_train_augmented[balanced_indices]

from sklearn.ensemble import GradientBoostingClassifier

model_no_data_aug = GradientBoostingClassifier()
model_no_data_aug.fit(X_audio_train_subset, y_audio_train_subset)
acc_no_data_aug = model_no_data_aug.score(X_audio_test, y_audio_test)
print(f"Accuracy without data augmentation: {acc_no_data_aug:.4f}")

model_data_aug = GradientBoostingClassifier()
model_data_aug.fit(X_audio_train_augmented_subset, y_audio_train_augmented_subset)
acc_data_aug = model_data_aug.score(X_audio_test, y_audio_test)
print(f"Accuracy with data augmentation: {acc_data_aug:.4f}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier

from visualization.show_metrics import plot_accuracy_vs_subset_size

subset_sizes = [100, 200, 300, 400, 500, 700, 1000]  # Different subset sizes to try
acc_no_aug_list = []
acc_aug_list = []

for subset_size in subset_sizes:
    samples_per_class = subset_size // 10
    
    # Get balanced subsets (no augmentation)
    balanced_indices = get_balanced_indices_by_targets(y_audio_train, samples_per_class, n_classes=10)
    X_train_sub = X_audio_train[balanced_indices]
    y_train_sub = y_audio_train[balanced_indices]
    
    # Get balanced subsets (with augmentation)
    balanced_indices_aug = get_balanced_indices_by_targets(y_audio_train_augmented, samples_per_class, n_classes=10)
    X_train_aug_sub = X_audio_train_augmented[balanced_indices_aug]
    y_train_aug_sub = y_audio_train_augmented[balanced_indices_aug]
    
    # Train and evaluate model without augmentation
    model_no_aug = GradientBoostingClassifier()
    model_no_aug.fit(X_train_sub, y_train_sub)
    acc_no_aug = model_no_aug.score(X_audio_test, y_audio_test)
    acc_no_aug_list.append(acc_no_aug)
    
    # Train and evaluate model with augmentation
    model_aug = GradientBoostingClassifier()
    model_aug.fit(X_train_aug_sub, y_train_aug_sub)
    acc_aug = model_aug.score(X_audio_test, y_audio_test)
    acc_aug_list.append(acc_aug)

plot_accuracy_vs_subset_size(
    subset_sizes=subset_sizes,
    acc_no_aug_list=acc_no_aug_list,
    acc_with_aug_list=acc_aug_list,
    title='Performance: Impact of Data Augmentation vs. Subset Size',
    save_path='accuracy_comparison.png'
)

In [ ]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1, 2, figsize=(20, 5))

# First tree - no augmentation
plot_tree(model_no_data_aug.estimators_[0, 0], filled=True,
          feature_names=[f"feat_{i}" for i in range(X_audio_train_subset.shape[1])],
          ax=axs[0])
axs[0].set_title("First Decision Tree - No Data Augmentation")

# First tree - with augmentation
plot_tree(model_data_aug.estimators_[0, 0], filled=True,
          feature_names=[f"feat_{i}" for i in range(X_audio_train_augmented_subset.shape[1])],
          ax=axs[1])
axs[1].set_title("First Decision Tree - With Data Augmentation")

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1, 2, figsize=(20, 5))

accuracies = [acc_no_data_aug, acc_data_aug]
labels = ['No Data Augmentation', 'With Data Augmentation']

axs[0].bar(labels, accuracies, color=['red', 'green'])
axs[0].set_ylim([0, 1])
axs[0].set_ylabel('Accuracy')
axs[0].set_title('Model Accuracy Comparison')
for i, acc in enumerate(accuracies):
    axs[0].text(i, acc + 0.02, f"{acc:.4f}", ha='center')
    
fi_no_aug = model_no_data_aug.feature_importances_
fi_aug = model_data_aug.feature_importances_

indices = np.arange(len(fi_no_aug))

axs[1].bar(indices - 0.2, fi_no_aug, width=0.4, label='No Augmentation')
axs[1].bar(indices + 0.2, fi_aug, width=0.4, label='With Augmentation')
axs[1].set_xlabel('Feature Index')
axs[1].set_ylabel('Feature Importance')
axs[1].set_title('Feature Importances Comparison')
axs[1].legend()
plt.show()

#### Images

In [ ]:
from torch import nn, optim

def train(model, dataloader, epochs=3):
    model.train()
    optimizer = optim.Adam(model.parameters())
    criterion = nn.CrossEntropyLoss()
    losses = []
    for epoch in range(epochs):
        for images, labels in dataloader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
    return losses

def test(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

In [ ]:
from data_util.MNIST.raw.mnist_loading import load_mnist_data
from models.simpleCNN import SimpleCNN
import torchvision.transforms as transforms

from visualization.show_images import visualize_pytorch_dataset_images
import albumentations as A

import numpy as np
import torch

class AlbumentationsTransform:
    def __init__(self, aug):
        self.aug = aug

    def __call__(self, img):
        # torchvision datasets give PIL.Image, convert to numpy
        img = np.array(img)
        augmented = self.aug(image=img)
        img = augmented["image"]
        # convert back to tensor
        if isinstance(img, np.ndarray):
            img = torch.from_numpy(img).float()
            if img.ndim == 2:  # add channel dim if grayscale
                img = img.unsqueeze(0)
            elif img.ndim == 3 and img.shape[2] in [1,3]:
                img = img.permute(2,0,1)  # HWC -> CHW
        return img



transform_no_aug = AlbumentationsTransform(A.Compose([
    A.Normalize(mean=0.5, std=0.5, max_pixel_value=255.0),
]))


transform_aug = AlbumentationsTransform(
    A.Compose([
        A.Rotate(limit=15, p=0.5),
        A.Affine(translate_percent=(0.1, 0.1), p=0.5),
        A.GaussianBlur(blur_limit=(3, 5), p=0.5, sigma_limit=(0.1, 2.0)),
        A.Normalize(mean=0.5, std=0.5, max_pixel_value=255.0),
    ])
)

subset_size = 100
num_classes = 10
epochs = 100 #WARNING - IF TOO LOW, MAY NOT WORK
samples_per_class = subset_size // 10

trainset_no_aug, trainloader_no_aug, testloader = load_mnist_data(train=True,data_augmentations=transform_no_aug,  subset_size=subset_size, )
trainset_aug, trainloader_aug, _ = load_mnist_data(train=True, data_augmentations=transform_aug, subset_size=subset_size)


visualize_pytorch_dataset_images(trainset_no_aug, "No Data Augmentation")
visualize_pytorch_dataset_images(trainset_aug, "With Data Augmentation")

In [ ]:
model_no_aug = SimpleCNN()
losses_no_aug = train(model_no_aug, trainloader_no_aug, epochs=epochs)
acc_no_aug = test(model_no_aug, testloader)

model_aug = SimpleCNN()
losses_aug = train(model_aug, trainloader_aug, epochs=epochs)
acc_aug = test(model_aug, testloader)

In [ ]:
from visualization.show_metrics import compare_two_losses

print(f"Accuracy without augmentation: {acc_no_aug:.4f}")
print(f"Accuracy with augmentation: {acc_aug:.4f}")

compare_two_losses(losses_no_aug, 'No Data Augmentation', losses_aug, "With Data Augmentation")

### Contrail Application

In [ ]:
import sys
sys.path.append("contrail-seg")
from train import train_contrail_network

transform_aug = A.Compose([
    A.Rotate(limit=15, p=0.5),
    A.Affine(translate_percent=(0.1, 0.1), p=0.5),
    A.Resize(128, 128),
    A.Normalize(mean=0.5, std=0.5, max_pixel_value=255.0),
])


train_contrail_network(transform_aug, 100, "dice", "base")